In [ ]:
#!pip install awswrangler

Defaulting to user installation because normal site-packages is not writeable


In [12]:
import awswrangler as wr

query = """
SELECT *
FROM glue_transformed
"""

df = wr.athena.read_sql_query(query, database="nyc_taxi_transformeddb")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4394088 entries, 0 to 4394087
Data columns (total 5 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   pickup_hour         string
 1   pickup_location_id  Int16 
 2   rides               Int16 
 3   year                string
 4   month               string
dtypes: Int16(2), string(3)
memory usage: 125.7 MB


In [ ]:
#!pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 7.2 MB/s eta 0:00:00


In [27]:
from sqlalchemy import create_engine

# Your RDS connection info
host = 'database-3.cudeiagum7dk.us-east-1.rds.amazonaws.com'
port = 5432
db = 'postgres'
user = 'postgres'
password = 'password'

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}')


In [28]:
df['year'] = df['year'].astype(int)
df['month'] = df['month'].astype(int)
df.to_sql('transformed_data', engine, if_exists='replace', index=False)

88

In [30]:
query_predictions = """
select * from model_lightgbm
"""
df_predictions = wr.athena.read_sql_query(query_predictions, database="taxi_predictions")
df_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26352 entries, 0 to 26351
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   prediction_datetime  26352 non-null  string
 1   predicted_rides      26352 non-null  Int64 
 2   location_id          26352 non-null  string
 3   year                 26352 non-null  string
 4   month                26352 non-null  string
 5   day                  26352 non-null  string
 6   hour                 26352 non-null  string
dtypes: Int64(1), string(6)
memory usage: 1.4 MB


In [31]:
df_predictions.head()

,prediction_datetime,predicted_rides,location_id,year,month,day,hour
0,2024-02-09 04:00:00,2,43,2024,02,09,4
1,2024-01-29 11:00:00,86,43,2024,01,29,11
2,2024-02-10 01:00:00,4,43,2024,02,10,1
3,2024-01-13 02:00:00,4,43,2024,01,13,2
4,2024-01-16 07:00:00,39,43,2024,01,16,7


In [32]:
import pandas as pd
df_predictions['year'] = df_predictions['year'].astype(int)
df_predictions['month'] = df_predictions['month'].astype(int)
df_predictions['day'] = df_predictions['day'].astype(int)
df_predictions['hour'] = df_predictions['hour'].astype(int)
df_predictions['predicted_rides'] = df_predictions['predicted_rides'].astype(int)
df_predictions['month'] = df_predictions['month'].astype(int)
df_predictions['location_id'] = df_predictions['location_id'].astype(int)
df_predictions.loc[df_predictions['prediction_datetime'].str.len() == 10, 'prediction_datetime'] += " 00:00:00"

# Now parse
df_predictions['prediction_datetime'] = pd.to_datetime(df_predictions['prediction_datetime'], errors='coerce')
df_predictions.to_sql('predictions_by_lightgbm', engine, if_exists='replace', index=False)

352

In [33]:
query_predictions_1 = """
select * from model_lightgbm_feature_engineering
"""
df_predictions_1 = wr.athena.read_sql_query(query_predictions_1, database="taxi_predictions")
df_predictions_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26352 entries, 0 to 26351
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   prediction_datetime  26352 non-null  string
 1   predicted_rides      26352 non-null  Int64 
 2   location_id          26352 non-null  string
 3   year                 26352 non-null  string
 4   month                26352 non-null  string
 5   day                  26352 non-null  string
 6   hour                 26352 non-null  string
dtypes: Int64(1), string(6)
memory usage: 1.4 MB


In [34]:
df_predictions_1['year'] = df_predictions_1['year'].astype(int)
df_predictions_1['month'] = df_predictions_1['month'].astype(int)
df_predictions_1['day'] = df_predictions_1['day'].astype(int)
df_predictions_1['hour'] = df_predictions_1['hour'].astype(int)
df_predictions_1['predicted_rides'] = df_predictions_1['predicted_rides'].astype(int)
df_predictions_1['month'] = df_predictions_1['month'].astype(int)
df_predictions_1['location_id'] = df_predictions_1['location_id'].astype(int)
df_predictions_1.loc[df_predictions_1['prediction_datetime'].str.len() == 10, 'prediction_datetime'] += " 00:00:00"

# Now parse
df_predictions_1['prediction_datetime'] = pd.to_datetime(df_predictions_1['prediction_datetime'], errors='coerce')
df_predictions_1.to_sql('predictions_by_lightgbm_with_feature_engineering', engine, if_exists='replace', index=False)

352